## ДЗ 4. Панков Викентий

## 1. Выровняем риды:
 ./hisat2-2.2.0/hisat2 refout -1 SRR453566_1.fastq -2 SRR453566_2.fastq > output.gff

## 2. Оценим количество экспрессирующихся генов.  
Получим статистику по количеству ридов на ген:  
 htseq-count output.sam genes.gtf  
Получим статистику по размерам генов:

Расчитаем TPM:

In [69]:
import pandas as pd

lengths = pd.read_csv("genes_lengths.txt", sep = "\t", index_col = False).set_index("gene").rename(columns={"median": "gene_length"})
readcounts = pd.read_csv("genes_stat.txt", sep = " ", index_col = False).set_index("gene")


In [66]:
read_length = 102
genes_number = len(readcounts)

merged = pd.merge(lengths[["gene_length"]], readcounts, left_index=True, right_index=True)
#merged = merged[merged.mapped_reads > 0]
merged["A"] = merged.mapped_reads*read_length*10**6/(merged.gene_length*genes_number)

merged["TPM"] = merged.A / merged.A.sum() * 10**6


merged

,gene_length,mapped_reads,A,TPM
gene,,,,
YJR023C,402,30,1067.443598,461.681867
YJR134C,2124,231,1555.633199,672.829591
YJL155C,1359,105,1105.145800,477.988511
YJL122W,528,179,4849.186012,2097.329783
YJL185C,882,20,324.347941,140.284286
...,...,...,...,...
YJR066W,7413,239,461.162130,199.458026
YJR144W,810,174,3072.656165,1328.959803
YJL156W-A,222,40,2577.251210,1114.691354


In [67]:
readcounts

,mapped_reads
gene,
15S_rRNA,2
21S_rRNA,22
HRA1,43
ICR1,37
LSR1,54
...,...
__no_feature,2141695
__ambiguous,31420
__too_low_aQual,587806


In [70]:
lengths

,mean,gene_length,longest_isoform,merged
gene,,,,
YJR023C,402,402,402,402
YJR134C,2124,2124,2124,2124
YJL155C,1359,1359,1359,1359
YJL122W,528,528,528,528
YJL185C,882,882,882,882
...,...,...,...,...
YJR066W,7413,7413,7413,7413
YJR144W,810,810,810,810
YJL156W-A,222,222,222,222


## 3. Соберём РНК-сборку и оценим качество:  
rnaspades -1 SRR453566_1.fastq -2 SRR453566_2.fastq -o rna_assembly  
python rnaquast/rnaQUAST.py -r ref.fa -c rna_assembly/transcripts.fasta -o rna_quast_output


Avg. aligned fraction                                  0.984                    
Avg. alignment length                                  1514.783                 
Avg. mismatches per transcript                         3.19